In [20]:
%pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.


## 새로운 Python 프로그램 생성

In [1]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os

# Gradio 환경 추가

In [3]:
%pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 63.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import gradio as gr
from PIL import Image, ImageDraw, ImageFont # pillow 라이브러리
import io # 이미지를 바이트 스트림으로 처리하기 위해 필요

prediction_endpoint = ""
prediction_key = ""
project_id = ""
model_name = ""

try:
    credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
    predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)
    print("Custom Vision predict가 성공적으로 초기화되었습니다.")
except Exception as e:
    print(f"Custom Vision predictor를 초기화 하는 중 에러가 발생했습니다: {e}")
    predictor = None # 초기화 실패 시 predictor는 None으로 설정

# 객체 감지 및 그리기 함수
def predict_and_draw(image: Image.Image, detection_threshold: float) -> Image.Image:
    """
    Gradio로부터 PIL 이미지를 받아 Custom Vision 예측 후 결과를 그립니다.

    Args:
        image: 입력 PIL Image 객체
        detection_threshold: 예측 결과 필터링을 위한 신뢰도 임계값 (0.0 ~ 1.0)

    Returns:
        예측 결과가 그려진 PIL Image 객체.
    """
    if predictor is None:
        # predictor 초기화 실패 시 에러 메시지를 이미지로 반환
        error_iamge = Image.new('RGB', (600, 400), color = (255, 200, 200))
        d = ImageDraw.Draw(error_iamge)
        d.text((10,10), "Error: Custom Vision Predictor가 초기화 되지 않았습니다.\n endpoint와 key 정보를 확인해주세요.", fill=(0,0,0))
        return error_iamge
    
    # PIL 이미지를 Custom Vision API 호출을 위한 바이트  스트림으로 변환
    # detect_image는 파일 객체나 바이트를 받습니다.
    img_byte_arr = io.BytesIO()
    # PIl 이미지를 특정 포맷으로 저장 (JPEG, PNG 등)
    # Custom Vision은 JPEG, PNG, BMP 등을 지원합니다. 원본 형식을 유지하거나 JPEG/PNG 사용
    try:
        # 원본 이미지의  형식을 그대로 사용하거나 기본값 사용
        image_format = image.format if image.format else 'PNG'
        image.save(img_byte_arr, format=image_format)
        img_byte_arr.seek(0) # 스트림의 시작으로 이동
        image_data = img_byte_arr.read()
    except Exception as e:
        print(f"이미지를 types로 변환하는 중 에러가 발생했습니다: {e}")
        error_iamge = Image.new('RGB', (600, 400), color = (255, 200, 200))
        d = ImageDraw.Draw(error_iamge)
        d.text((10,10), "Error: 이미지 포맷을 처리하는 중 문제가 발생했습니다: {e}", fill=(0,0,0))
        return error_iamge

    try:
        # Custom Vision API 호출
        # detection_threshold 파라미터를 사용하여 서비스 측에서 필터링
        results = predictor.detect_image(
            project_id,
            model_name,
            image_data,
            detection_threshold=detection_threshold # 여기서 임계값을 적용
        )
    except Exception as e:
        print(f"Custom Vision prediction을 호출하는 중 에러가 발생했습니다: {e}")

    # 결과 이미지에 바운딩 박스와 라벨 그리기
    # 원본 이미지에 직접 그리기 위해 복사본 사용 권장
    # Gradio 추력은 새로운 이미지를 반환하면 되므로 복사본이 안전합니다.
    output_image = image.copy()
    draw = ImageDraw.Draw(output_image)
    width, height = output_image.size

    # 선 두께 계산 (이미지 크기에 따라 조절)
    line_width = max(2, int(width / 200)) # 최소 2픽셀, 이미지 폭의 1/200 정도

    # 라벨 텍스트를 위한 기본 폰트 로드 (PIL에서 기본 제공)
    try:
        font = ImageFont.load_default()
    except Exception as e:
        print(f"경고: 폰트를 불러올 수 없습니다. 기본 PIL 폰트를 사용하겠습니다: {e}")
        font = ImageFont.load_default()

    # Custom Vision API에서 반환된 예측 결과 순회
    for prediction in results.predictions:
        # detection_threshold를 사용하여 probability 체크
        if prediction.probability > detection_threshold:
            # 좌표 스케일링 (API 결과는 0~1 사이의 값)
            left = prediction.bounding_box.left * width
            top = prediction.bounding_box.top * height
            box_width = prediction.bounding_box.width * width
            box_height = prediction.bounding_box.height * height

            # 바운딩 박스 좌표 계산
            box_coords = (left, top, left + box_width, top + box_height)
            # 색상 설정: 마젠타
            color = 'magenta'

            # 바운딩 박스 그리기
            draw.rectangle(box_coords, outline=color, width=line_width)

            # 라벨 텍스트 생성 (태그 이름 및 신뢰도)
            label_text = f"{prediction.tag_name}: {prediction.probability*100:.1f}"

            # 라벨 텍스트 위치 계산 (바운딩 박스 상단 약간 위)
            # 텍스트 크기를 대략적으로 계산하여 위치 조정
            try:
                text_bbox = draw.textbbox((left, top), label_text, font=font)
                text_width = text_bbox[2] - text_bbox[0]
                text_height = text_bbox[3] - text_bbox[1]
            except AttributeError:
                print(f"AttributeError가 발생했습니다.")
                text_width, text_height = draw.textsize(label_text, font=font)

            text_x = left
            text_y = top - text_height -5 # 상단에서 텍스트 높이 + 약간의 여백만큼 위로
            
            # 라벨 텍스트 그리기
            draw.text((text_x, text_y), label_text, fill=color, font=font) # 텍스트 색상

    # Matplotlib 대신 PIL Image 객체를 직접 반환
    return output_image

iface = gr.Interface(
    fn=predict_and_draw,
    inputs=[
        gr.Image(type="pil", label="이미지를 업로드하세요"), # 이미지 입력 (PIL Image 객체로 받음)
        gr.Slider(minimum=0, maximum=1, value=0.5, label="예측 신뢰도 임계값 (Detection Threshold)") # 슬라이더 입력
    ],
    outputs=gr.Image(type="pil", label="감지된 객체"), # 이미지 출력 (PIl Image 객체 반환)
    title="Azure Custom Vision 과일 객체 탐지 데모",
    description="이미지를 업로드하고 임계값을 조절하여 Azure Custom Vision 모델의 객체 감지 결과를 확인하세요"
)

iface.launch(share=True)

Custom Vision predict가 성공적으로 초기화되었습니다.
* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://f0b00be6a61347ca46.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
